In [38]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [39]:
cd /content/gdrive/MyDrive/Thesis/ASD

/content/gdrive/MyDrive/Thesis/ASD


In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from functools import reduce
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import time
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import sys
import pickle
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold, StratifiedKFold
import torch.optim as optim
from sklearn.metrics import confusion_matrix
import functools
import numpy.ma as ma # for masked arrays
import glob
import random
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from tqdm.notebook import tqdm
from itertools import groupby
import sklearn

In [ ]:
!pip install captum

In [ ]:
from sklearn.model_selection import train_test_split
from captum.attr import IntegratedGradients
from captum.attr import Saliency
from captum.attr import DeepLift
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz
from captum.attr import Saliency
import torchvision

### Helper functions for computing correlations

In [41]:
def get_corr_data(df):
              
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(df.T))
        mask = np.invert(np.tri(corr.shape[0], k=-1, dtype=bool))
        m = ma.masked_where(mask == 1, mask)
        return ma.masked_where(m, corr).compressed()
        
def get_corr_matrix(filename,data_path):
    # returns correlation matrix
    for file in os.listdir(data_path):
        if file.startswith(filename):
            df = pd.read_csv(os.path.join(data_path, file), sep='\t')
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(df.T))
        return corr

def confusion(g_turth,predictions):
    tn, fp, fn, tp = confusion_matrix(g_turth,predictions).ravel()
    accuracy = (tp+tn)/(tp+fp+tn+fn)
    sensitivity = (tp)/(tp+fn)
    specificty = (tn)/(tn+fp)
    return accuracy,sensitivity,specificty

## Loading the data 

In [42]:
def get_key(filename):
    f_split = filename.split('_')
    if f_split[3] == 'rois':
        key = '_'.join(f_split[0:3]) 
    else:
        key = '_'.join(f_split[0:2])
    return key

In [43]:
data_df = pd.read_csv('./Phenotypes/Phenotypic_V1_0b_preprocessed949.csv', encoding= 'unicode_escape')
data_df.DX_GROUP = data_df.DX_GROUP.map({1: 1, 2:0})
print('Length of data frame : ', len(data_df))
print(data_df.head())

Length of data frame :  949
   Unnamed: 0  SUB_ID  X  subject SITE_ID       FILE_ID  DX_GROUP  DSM_IV_TR  \
0           0   50003  2    50003    PITT  Pitt_0050003         1          1   
1           1   50004  3    50004    PITT  Pitt_0050004         1          1   
2           2   50006  5    50006    PITT  Pitt_0050006         1          1   
3           3   50007  6    50007    PITT  Pitt_0050007         1          1   
4           4   50009  8    50009    PITT  Pitt_0050009         1          1   

   AGE_AT_SCAN  SEX  ... qc_notes_rater_1  qc_anat_rater_2  \
0        24.45    1  ...              NaN               OK   
1        19.09    1  ...              NaN               OK   
2        13.37    1  ...              NaN               OK   
3        17.78    1  ...              NaN               OK   
4        33.86    1  ...              NaN               OK   

   qc_anat_notes_rater_2  qc_func_rater_2       qc_func_notes_rater_2  \
0                    NaN               OK    

In [ ]:
data_path = './Datasets/CPAC/rois_cc200/*.1D'
fpaths = glob.glob(data_path)
print('Number of Subjects available: ', len(fpaths))

Number of Subjects available:  1035


In [ ]:
all_corr = {}

for i,path in enumerate(fpaths):

    # Extracting SFC
    fname = path.split('/')[-1]
    key = fname.split('_')[0]
    x = np.loadtxt(path)
    x = np.array(x, dtype = 'float32')
    corr = get_corr_data(x)

    # Extracting corresponding labels
    y = data_df[data_df['SUB_ID'] == int(key)]['DX_GROUP']

    all_corr[key] = (corr,y)

print('Length of correlations vector : ', len(all_corr))
pickle.dump(all_corr, open('./data/SFC_CC200.pkl', 'wb'))

Length of correlations vector :  866


In [44]:
all_corr = pickle.load(open('./data/SFC_CC200.pkl', 'rb'))
flist = np.array(list(all_corr.keys()))
labels = np.array([all_corr[f][1] for f in flist], dtype = 'uint8')
print('Length of Input subjects : ', len(flist))
print('Length of Output subjects : ', len(labels))

Length of Input subjects :  949
Length of Output subjects :  949


# DataLoader

In [45]:
class ASDDataset(Dataset):
    def __init__(self, all_corr, subjects):
        self.corr = all_corr
        self.subjects = subjects
        pass
    def __getitem__(self,idx):
        return torch.tensor(self.corr[self.subjects[idx]][0],dtype=torch.float),torch.tensor(self.corr[self.subjects[idx]][1],dtype=torch.float)
        pass
    def __len__(self):
        return len(self.subjects)
        pass

# Network 

In [46]:
# Auto Encoder and Classifier
class Network(nn.Module):
    def __init__(self, num_inputs = 19900, num_latent = 200):
        super(Network, self).__init__()
        
        self.num_latent = num_latent
        self.num_inputs = num_inputs
        
        self.fc_encoder = nn.Sequential (
                nn.Linear(self.num_inputs,4096),
                nn.Tanh(),
                nn.Linear(4096,1024),
                nn.Tanh())
        
        self.fc_decoder = nn.Sequential (
                nn.Linear(1024,4096),
                nn.Tanh(),
                nn.Linear(4096,self.num_inputs),
                nn.Tanh())
         
        self.classifier = nn.Sequential (
            nn.Dropout(p=0.25),
            nn.Linear(1024, 1),
        )

        self.sigmoid = nn.Sigmoid()           
         
    def forward(self, x, eval_classifier = True):

        x = self.fc_encoder(x)
        if eval_classifier:
            x_logit = self.classifier(x)   #   .squeeze(1)
            x_logit = self.sigmoid(x_logit)
            return x_logit 

        x = self.fc_decoder(x)        
        return x

# Defining training and testing functions

In [51]:
def train(model, criterion, data_loader, mode='clf'):
    model.train()
    clf_loss = []
    ae_loss = []
    
    if mode == 'clf':
        final_targets = []
        final_predictions = []
    else:
        final_targets = None
        final_predictions = None    
    
    for (inputs, targets) in data_loader :
        # if len(batch_x) != batch_size:           # Check 1
        #     continue

        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        if mode == 'ae':        
            reconstructed = model(inputs, False)
            loss_ae = criterion(reconstructed, inputs) / len(inputs)              
            loss_total = loss_ae
            loss_ae_np = loss_ae.detach().cpu().numpy()
            ae_loss.append(loss_ae_np)           

        elif mode == 'clf':
            logits = model(inputs, True)
            logits = np.squeeze(logits, 1)
            loss_clf = criterion(logits, targets)
            proba = logits.detach().cpu().numpy()
            # proba = torch.sigmoid(logits).detach().cpu().numpy().    # Make sure sigmoid or softmax is present in the architecture function, otherwise uncomment this.
            predictions = np.round(proba)           
            final_targets.append(targets.detach().cpu().numpy())
            final_predictions.append(predictions)
            
            loss_total = loss_clf
            loss_clf_np = loss_clf.detach().cpu().numpy()           
            clf_loss.append(loss_clf_np)
            
        loss_total.backward()
        optimizer.step()
    
    if (final_targets is not None) and (final_predictions is not None):
        final_targets = np.concatenate(final_targets)
        final_predictions = np.concatenate(final_predictions)
        train_accuracy = np.mean(final_targets == final_predictions)

        return np.mean(clf_loss), train_accuracy
    else:
        return np.mean(ae_loss), None


def test(model, criterion, data_loader, mode = 'clf'):

    clf_loss = []
    ae_loss = []
    
    if mode == 'clf':
        final_targets = []
        final_predictions = []
    else:
        final_targets = None
        final_predictions = None    
    
    with torch.no_grad():
        model.eval()
        for (inputs, targets) in data_loader :
            # if len(batch_x) != batch_size:           # Check 1
            #     continue

            inputs, targets = inputs.to(device), targets.to(device)
            if mode == 'ae':        
                reconstructed = model(inputs, False)
                loss_ae = criterion_ae(reconstructed, inputs) / len(inputs)           
                loss_total = loss_ae
                loss_ae_np = loss_ae.detach().cpu().numpy()
                ae_loss.append(loss_ae_np)           

            if mode == 'clf':
                logits = model(inputs, True)
                logits = np.squeeze(logits, 1)
                loss_clf = criterion_clf(logits, targets)
                proba = logits.detach().cpu().numpy()
                # proba = torch.sigmoid(logits).detach().cpu().numpy().    # Make sure sigmoid or softmax is present in the architecture function, otherwise uncomment this.
                predictions = np.round(proba)           
                final_targets.append(targets.detach().cpu().numpy())
                final_predictions.append(predictions)
                
                loss_total = loss_clf
                loss_clf_np = loss_clf.detach().cpu().numpy()           
                clf_loss.append(loss_clf_np)
              
    
    if (final_targets is  None) and (final_predictions is None):
        return np.mean(ae_loss), None
    
    final_targets = np.concatenate(final_targets)
    final_predictions = np.concatenate(final_predictions)
    mlp_acc, mlp_sens, mlp_spef = confusion(final_targets, final_predictions)
    metrics_dict = {'accuracy': np.round(mlp_acc, 4), 
                    'senstivity' : np.round(mlp_sens,4), 
                    'specificity' : np.round(mlp_spef,4), 
                    'loss' : np.round(np.mean(clf_loss),4)}               
    return  metrics_dict  

In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
def attribute_image_features(algorithm, inputs):
    model.zero_grad()
    model.eval()
    tensor_attributions = algorithm.attribute(inputs = inputs, target = 0, return_convergence_delta=True)  
    return tensor_attributions

# ASD 2 Layer Model Training

In [49]:
# Define Parameters
p_fold = 10
batch_size = 16
lr_ae, lr_clf = 0.0001, 0.0001
ae_epochs, clf_epochs = 50, 50     # 50, 50
weight_decay_ae, weight_decay_clf = 0.05, 0.05
n_inputs, n_latent = 19900, 512  # Automate this according to the atlas 

In [52]:
crossval_acc, crossval_sen, crossval_spec, crossval_loss, attributions = [], [], [], [], [] 
all_folds_splits = {}
kf = StratifiedKFold(n_splits = p_fold, random_state = 1, shuffle = True)

start = time.time()
for fold,(train_index, test_index) in enumerate(kf.split(flist, labels)):

    train_subjects, test_subjects = flist[train_index],flist[test_index]
    train_labels = labels[train_index]   
    train_subjects, val_subjects, train_labels, val_labels = train_test_split(train_subjects, train_labels, 
                                                      test_size = 0.25, random_state = 42, stratify = train_labels)
    
    print('Number of train subjects : ', len(train_subjects))
    print('Number of val subjects : ', len(val_subjects))
    print('Number of test subjects : ', len(test_subjects))

    fold_splits_dict = {} 
    fold_splits_dict['train'] = train_subjects
    fold_splits_dict['val'] = val_subjects
    fold_splits_dict['test'] = test_subjects

    all_folds_splits[fold] = fold_splits_dict
    verbose = (True if (fold == 0) else False)
   
    train_dataset = ASDDataset(all_corr, train_subjects)
    val_dataset = ASDDataset(all_corr, val_subjects)
    test_dataset = ASDDataset(all_corr, test_subjects)
    
    train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
    val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False)
    test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)                           

    model = Network(num_inputs = n_inputs, num_latent = n_latent)
    model = model.to(device)

    criterion_ae = nn.MSELoss(reduction='sum')         
    optimizer = optim.Adam(model.parameters(), lr = lr_ae, weight_decay = weight_decay_ae)          
    best_ae_model, count = None, 1
    best_ae_loss = sys.float_info.max
    
    print("Auto Encoder training Started-----------")
    for epoch in range(1, ae_epochs+1):

        print(f'Epoch {epoch}/{ae_epochs}')
        ae_train_loss, _ = train(model, criterion_ae, train_dataloader, mode = 'ae')
        print(f'AE Train loss: {(ae_train_loss):.4f}')

        ae_val_loss, _ = test(model, criterion_ae, val_dataloader, mode = 'ae')
        print(f'AE Val loss: {(ae_val_loss):.4f}')

        if(ae_val_loss < best_ae_loss):     # Early Stopping 
            best_ae_model = model
            best_ae_loss = ae_val_loss
            count = 1
        else:
            count += 1     
        if(count == 10):  # Criteria
            break
              
    best_clf_model, best_clf_acc, count = None, 0.0, 1
    model = best_ae_model
    criterion_clf = nn.BCELoss()      
    optimizer = optim.Adam(model.parameters(), lr = lr_clf, weight_decay = weight_decay_clf)

    print("Classifier training Started-----------")
    for epoch in range(1, clf_epochs+1):

        print(f'Epoch {epoch}/{clf_epochs}')
        clf_train_loss, train_acc = train(model, criterion_clf, train_dataloader, mode='clf')
        print(f'CLF Train loss: {(clf_train_loss):.4f}, Train Accuracy: {(train_acc):.4f}')

        val_metrics = test(model, criterion_clf, val_dataloader, mode='clf')
        print(f'CLF Val loss: {(val_metrics["loss"]):.4f}, Validation Accuracy: {(val_metrics["accuracy"]):.4f}')

        if(val_metrics['accuracy'] > best_clf_acc):    # Early Stopping Criteria
            best_clf_model = model
            best_clf_acc = val_metrics['accuracy']
            count = 1
        else:
            count += 1
        if(count == 10):
            break        

    metrics_dict = test(best_clf_model, criterion_clf, test_dataloader, mode = 'clf')

    print(f'Fold {fold+1}/{p_fold}')
    print(f'{metrics_dict}')
    print("--------------------------------------------")
    
    # torch.save(best_clf_model.state_dict(), f'./data/Weights/Fold_{kk+1}.pth').    # To save the weights
    # print(f'Fold {kk+1} weights are saved')

    crossval_acc.append(metrics_dict['accuracy'])
    crossval_sen.append(metrics_dict['senstivity'])
    crossval_spec.append(metrics_dict['specificity'])
    crossval_loss.append(metrics_dict['loss'])

print(f'Average Value after 10 Folds')
print(f'Accuracy: {np.round(np.mean(crossval_acc),4)}, Senstivity: {np.round(np.mean(crossval_sen),4)}, Specificity: {np.round(np.mean(crossval_spec),4)}, Loss: {np.round(np.mean(crossval_loss),4)}')
# pickle.dump(df, open('./data/AllFoldssubjects.pkl', 'wb'))
print(f'Total time taken : {time.time()-start}')

Number of train subjects :  640
Number of val subjects :  214
Number of test subjects :  95
Auto Encoder training Started-----------
Epoch 1/50
AE Train loss: 774.6885
AE Val loss: 698.0565
Epoch 2/50
AE Train loss: 632.6107
AE Val loss: 639.1536
Epoch 3/50
AE Train loss: 556.9248
AE Val loss: 605.7745
Epoch 4/50
AE Train loss: 500.7336
AE Val loss: 586.5453
Epoch 5/50
AE Train loss: 454.2633
AE Val loss: 575.3898
Epoch 6/50
AE Train loss: 413.2725
AE Val loss: 562.2501
Epoch 7/50
AE Train loss: 377.2842
AE Val loss: 546.5216
Epoch 8/50
AE Train loss: 341.6988
AE Val loss: 539.1572
Epoch 9/50
AE Train loss: 312.0334
AE Val loss: 532.9943
Epoch 10/50
AE Train loss: 286.8349
AE Val loss: 528.0292
Epoch 11/50
AE Train loss: 262.6194
AE Val loss: 521.5522
Epoch 12/50
AE Train loss: 244.6883
AE Val loss: 522.6520
Epoch 13/50
AE Train loss: 227.4125
AE Val loss: 514.4360
Epoch 14/50
AE Train loss: 209.9804
AE Val loss: 511.0320
Epoch 15/50
AE Train loss: 194.7498
AE Val loss: 517.5863
Epoch 

In [53]:
crossval_acc, crossval_sen, crossval_spec, crossval_loss, attributions = [], [], [], [], [] 
all_folds_splits = {}
kf = StratifiedKFold(n_splits = p_fold, random_state = 1, shuffle = True)

start = time.time()
for fold,(train_index, test_index) in enumerate(kf.split(flist, labels)):

    train_subjects, test_subjects = flist[train_index],flist[test_index]
    train_labels = labels[train_index]   
    train_subjects, val_subjects, train_labels, val_labels = train_test_split(train_subjects, train_labels, 
                                                      test_size = 0.20, random_state = 42, stratify = train_labels)
    
    print('Number of train subjects : ', len(train_subjects))
    print('Number of val subjects : ', len(val_subjects))
    print('Number of test subjects : ', len(test_subjects))

    fold_splits_dict = {} 
    fold_splits_dict['train'] = train_subjects
    fold_splits_dict['val'] = val_subjects
    fold_splits_dict['test'] = test_subjects

    all_folds_splits[fold] = fold_splits_dict
    verbose = (True if (fold == 0) else False)
   
    train_dataset = ASDDataset(all_corr, train_subjects)
    val_dataset = ASDDataset(all_corr, val_subjects)
    test_dataset = ASDDataset(all_corr, test_subjects)
    
    train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
    val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False)
    test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)                           

    model = Network(num_inputs = n_inputs, num_latent = n_latent)
    model = model.to(device)

    criterion_ae = nn.MSELoss(reduction='sum')         
    optimizer = optim.Adam(model.parameters(), lr = lr_ae, weight_decay = weight_decay_ae)          
    best_ae_model, count = None, 1
    best_ae_loss = sys.float_info.max
    
    print("Auto Encoder training Started-----------")
    for epoch in range(1, ae_epochs+1):

        print(f'Epoch {epoch}/{ae_epochs}')
        ae_train_loss, _ = train(model, criterion_ae, train_dataloader, mode = 'ae')
        print(f'AE Train loss: {(ae_train_loss):.4f}')

        ae_val_loss, _ = test(model, criterion_ae, val_dataloader, mode = 'ae')
        print(f'AE Val loss: {(ae_val_loss):.4f}')

        if(ae_val_loss < best_ae_loss):     # Early Stopping 
            best_ae_model = model
            best_ae_loss = ae_val_loss
            count = 1
        else:
            count += 1     
        if(count == 10):  # Criteria
            break
              
    best_clf_model, best_clf_acc, count = None, 0.0, 1
    model = best_ae_model
    criterion_clf = nn.BCELoss()      
    optimizer = optim.Adam(model.parameters(), lr = lr_clf, weight_decay = weight_decay_clf)

    print("Classifier training Started-----------")
    for epoch in range(1, clf_epochs+1):

        print(f'Epoch {epoch}/{clf_epochs}')
        clf_train_loss, train_acc = train(model, criterion_clf, train_dataloader, mode='clf')
        print(f'CLF Train loss: {(clf_train_loss):.4f}, Train Accuracy: {(train_acc):.4f}')

        val_metrics = test(model, criterion_clf, val_dataloader, mode='clf')
        print(f'CLF Val loss: {(val_metrics["loss"]):.4f}, Validation Accuracy: {(val_metrics["accuracy"]):.4f}')

        if(val_metrics['accuracy'] > best_clf_acc):    # Early Stopping Criteria
            best_clf_model = model
            best_clf_acc = val_metrics['accuracy']
            count = 1
        else:
            count += 1
        if(count == 10):
            break        

    metrics_dict = test(best_clf_model, criterion_clf, test_dataloader, mode = 'clf')

    print(f'Fold {fold+1}/{p_fold}')
    print(f'{metrics_dict}')
    print("--------------------------------------------")
    
    # torch.save(best_clf_model.state_dict(), f'./data/Weights/Fold_{kk+1}.pth').    # To save the weights
    # print(f'Fold {kk+1} weights are saved')

    crossval_acc.append(metrics_dict['accuracy'])
    crossval_sen.append(metrics_dict['senstivity'])
    crossval_spec.append(metrics_dict['specificity'])
    crossval_loss.append(metrics_dict['loss'])

print(f'Average Value after 10 Folds')
print(f'Accuracy: {np.round(np.mean(crossval_acc),4)}, Senstivity: {np.round(np.mean(crossval_sen),4)}, Specificity: {np.round(np.mean(crossval_spec),4)}, Loss: {np.round(np.mean(crossval_loss),4)}')
# pickle.dump(df, open('./data/AllFoldssubjects.pkl', 'wb'))
print(f'Total time taken : {time.time()-start}')

Number of train subjects :  683
Number of val subjects :  171
Number of test subjects :  95
Auto Encoder training Started-----------
Epoch 1/50
AE Train loss: 778.7787
AE Val loss: 690.0318
Epoch 2/50
AE Train loss: 631.7645
AE Val loss: 628.2238
Epoch 3/50
AE Train loss: 554.9092
AE Val loss: 594.4934
Epoch 4/50
AE Train loss: 500.8165
AE Val loss: 575.5294
Epoch 5/50
AE Train loss: 455.1542
AE Val loss: 564.6990
Epoch 6/50
AE Train loss: 415.4597
AE Val loss: 547.8933
Epoch 7/50
AE Train loss: 376.4567
AE Val loss: 545.7291
Epoch 8/50
AE Train loss: 343.7767
AE Val loss: 531.0880
Epoch 9/50
AE Train loss: 314.3024
AE Val loss: 520.2012
Epoch 10/50
AE Train loss: 287.2277
AE Val loss: 518.3170
Epoch 11/50
AE Train loss: 264.0032
AE Val loss: 509.0962
Epoch 12/50
AE Train loss: 245.9659
AE Val loss: 505.0529
Epoch 13/50
AE Train loss: 227.2921
AE Val loss: 502.5721
Epoch 14/50
AE Train loss: 210.4479
AE Val loss: 498.9919
Epoch 15/50
AE Train loss: 195.3605
AE Val loss: 496.9584
Epoch 

# Visualization

In [ ]:
def attribute_image_features(algorithm, inputs, target = 1):
    model.zero_grad()
    model.eval()
    tensor_attributions = algorithm.attribute(inputs = inputs, target = target, return_convergence_delta=True)  
    return tensor_attributions

In [ ]:
attributions = []
all_folds = pickle.load(open('./data/AllFoldssubjects.pkl', 'rb'))

for fold in range(10):
    fold_weights = torch.load(f'data/Weights/Fold_{fold+1}.pth', map_location=torch.device('cpu'))
    best_clf_model = MTAutoEncoder(tied = False, num_inputs = 19900, num_latent = 512, use_dropout = True) 
    best_clf_model.load_state_dict(fold_weights)
    best_clf_model = best_clf_model.to('cpu')

    test_subjects = all_folds[fold]['test']
    x_asd, y_asd = [], []
    for sample in test_subjects : 
        if(all_corr[sample][1] == 1):
          x_asd.append(all_corr[sample][0])
          y_asd.append(all_corr[sample][1])
    print('Number of ASD subjects in test set : ', len(x_asd))

    x_asd = torch.tensor(x_asd, dtype=torch.float)
    y_asd = np.array(y_asd)  

    y_asd_pred = best_clf_model(x_asd)
    y_asd_pred = y_asd_pred.detach().cpu().numpy()
    y_asd_pred = np.round(y_asd_pred)
    y_asd_pred = np.squeeze(y_asd_pred, axis = 1)

    right_indices = np.where(y_asd_pred == 1)
    x_asd_ig = x_asd[right_indices]
    print('Number of correctly predicted ASD subjects : ', len(x_asd_ig))

    ig_asd = IntegratedGradients(best_clf_model)        
    grads_asd, delta_asd = attribute_image_features(ig_asd, inputs = x_asd_ig, target = 0)                     
    grads_asd = torch.mean(grads_asd, axis = 0)
    grads_asd = np.array(grads_asd)
    attributions.append(grads_asd)
                            
attributions = np.array(attributions)
# np.save('./data/IG_Attributions.npy', attributions)
print("Attributions shape : ", attributions.shape)

Number of ASD samples in test set :  42


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


Number of correctly predicted ASD samples :  28
Number of ASD samples in test set :  42
Number of correctly predicted ASD samples :  23
Number of ASD samples in test set :  42
Number of correctly predicted ASD samples :  26
Number of ASD samples in test set :  42
Number of correctly predicted ASD samples :  28
Number of ASD samples in test set :  42
Number of correctly predicted ASD samples :  31
Number of ASD samples in test set :  42
Number of correctly predicted ASD samples :  32
Number of ASD samples in test set :  42
Number of correctly predicted ASD samples :  29
Number of ASD samples in test set :  42
Number of correctly predicted ASD samples :  32
Number of ASD samples in test set :  42
Number of correctly predicted ASD samples :  23
Number of ASD samples in test set :  41
Number of correctly predicted ASD samples :  19
Attributions shape :  (10, 19900)


In [ ]:
rois_count = {}    # {roi : number of times it is repeated in all 10 folds}

for grads_asd in attributions :
  
    attr_vals_asd = grads_asd.copy()
    thresh = np.percentile(attr_vals_asd, 99)
    attr_vals_asd = np.where(attr_vals_asd > thresh,  1 , 0) # check1
    corr_matrix_asd = np.zeros((200,200))
    corr_matrix_asd[np.triu_indices(200, 1)] = attr_vals_asd
    print('Number of unique elements in corr_matrix : ', np.unique(corr_matrix_asd, return_counts = True))

    max_sum_rows = np.sum(corr_matrix_asd, axis = 1)      # check 2
    top_indices = np.argsort(max_sum_rows)            # Max value indices
    top_values = max_sum_rows[top_indices]      # Max values

    top20_indices  = top_indices[-20 : ]
    top20_values = top_values[-20 : ]

    print('Most repeated ROIS in ASD (Not index values): ', top20_indices + 1)
    print('Number of times ROIS repeated in ASD : ', top20_values)
    
    for index, roi in enumerate(top_indices): 
        if(roi in rois_count):
            rois_count[roi] += top_values[index]
        else : 
          rois_count[roi] = top_values[index]        

rois_count_sorted = dict(sorted(rois_count.items(), key = lambda x : x[1], reverse = True))
print(rois_count_sorted)

Number of unique elements in corr_matrix :  (array([0., 1.]), array([39801,   199]))
Most repeated ROIS in ASD (Not index values):  [154  30 107 105 106  65  31  10  63  28  71  21  32  59 117  57  64  49
  94  55]
Number of times ROIS repeated in ASD :  [3. 3. 3. 3. 3. 4. 4. 4. 4. 4. 5. 5. 5. 6. 6. 6. 6. 6. 7. 8.]
Number of unique elements in corr_matrix :  (array([0., 1.]), array([39801,   199]))
Most repeated ROIS in ASD (Not index values):  [107  21  33  28  90  93  45  71 154 130  49 106  59  46  32  55  30  17
  77  64]
Number of times ROIS repeated in ASD :  [3. 3. 3. 3. 3. 3. 3. 4. 4. 4. 4. 5. 5. 5. 5. 6. 6. 6. 6. 7.]
Number of unique elements in corr_matrix :  (array([0., 1.]), array([39801,   199]))
Most repeated ROIS in ASD (Not index values):  [128  94 130  93  55   8  66  31  16  36  32  45  49  57  63  64  42  46
  30  59]
Number of times ROIS repeated in ASD :  [3. 3. 3. 3. 3. 3. 3. 3. 3. 4. 4. 4. 4. 4. 4. 4. 4. 5. 5. 6.]
Number of unique elements in corr_matrix :  (arra

In [ ]:
print(rois_count_sorted)

{63: 61.0, 58: 56.0, 31: 49.0, 48: 48.0, 54: 46.0, 29: 44.0, 93: 40.0, 45: 37.0, 20: 37.0, 16: 35.0, 27: 35.0, 116: 34.0, 105: 33.0, 14: 29.0, 52: 28.0, 62: 28.0, 70: 26.0, 89: 25.0, 44: 25.0, 7: 24.0, 83: 24.0, 56: 24.0, 35: 23.0, 32: 22.0, 41: 22.0, 88: 22.0, 64: 22.0, 46: 21.0, 73: 21.0, 4: 21.0, 10: 21.0, 15: 20.0, 30: 20.0, 72: 19.0, 92: 19.0, 34: 19.0, 106: 19.0, 9: 19.0, 76: 18.0, 86: 17.0, 12: 17.0, 38: 17.0, 163: 17.0, 65: 17.0, 43: 17.0, 3: 16.0, 165: 16.0, 113: 16.0, 11: 16.0, 118: 16.0, 33: 16.0, 109: 16.0, 129: 16.0, 104: 16.0, 167: 15.0, 90: 15.0, 130: 15.0, 82: 15.0, 127: 15.0, 74: 15.0, 153: 15.0, 161: 14.0, 77: 13.0, 2: 13.0, 128: 12.0, 98: 11.0, 91: 11.0, 17: 11.0, 154: 11.0, 85: 10.0, 169: 10.0, 67: 10.0, 139: 10.0, 125: 10.0, 102: 10.0, 146: 10.0, 66: 9.0, 36: 9.0, 23: 9.0, 99: 9.0, 111: 8.0, 24: 8.0, 94: 8.0, 117: 7.0, 101: 7.0, 189: 7.0, 158: 7.0, 28: 7.0, 164: 7.0, 84: 7.0, 186: 6.0, 187: 6.0, 19: 6.0, 180: 6.0, 182: 6.0, 61: 6.0, 0: 5.0, 134: 5.0, 103: 5.0, 47: 

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [ ]:
cc200_labels = pd.read_csv('./data/CC200_ROI_labels.csv')
# cc200_labels.head(5)
print('ASD Associated Regions : ')
rois = [64, 59, 32, 49, 55, 30, 94, 46]
asd_rois = cc200_labels[cc200_labels['ROI number'].isin(rois)]
display(asd_rois)

ASD Associated Regions : 


,ROI number,volume,center of mass,Dosenbach,AAL,Eickhoff-Zilles,Talairach-Tournoux,Harvard-Oxford
29,30,194,(2.9;-28.0;-36.0),"[""None"": 1.00]","[""None"": 1.00]","[""None"": 1.00]","[""None"": 1.00]","[""None"": 1.00]"
31,32,170,(43.5;9.9;-36.2),"[""None"": 1.00]","[""Temporal_Pole_Mid_R"": 0.59][""Temporal_Inf_R"": 0.35]","[""Right Medial Temporal Pole"": 0.61][""Right Inferior Temporal Gyrus"": 0.26][""None"": 0.12]","[""Right Middle Temporal Gyrus"": 0.44][""None"": 0.31][""Right Superior Temporal Gyrus"": 0.25]","[""Right Temporal Pole"": 0.96]"
45,46,227,(1.9;-38.2;32.2),"[""None"": 0.84]","[""Cingulum_Mid_R"": 0.33][""Cingulum_Post_L"": 0.23][""Cingulum_Mid_L"": 0.18][""Cingulum_Post_R"": 0.18]","[""Left Posterior Cingulate Cortex"": 0.26][""Right Middle Cingulate Cortex"": 0.24][""Right Posterior Cingulate Cortex"": 0.24][""Left Middle Cingulate Cortex"": 0.14]","[""Right Cingulate Gyrus"": 0.44][""Left Cingulate Gyrus"": 0.26][""Right Posterior Cingulate"": 0.14][""Left Posterior Cingulate"": 0.13]","[""Right Cingulate Gyrus; posterior division"": 0.74][""Left Cingulate Gyrus; posterior division"": 0.25]"
48,49,222,(61.9;-21.1;-15.6),"[""None"": 0.94]","[""Temporal_Mid_R"": 0.64][""Temporal_Inf_R"": 0.34]","[""Right Inferior Temporal Gyrus"": 0.53][""Right Middle Temporal Gyrus"": 0.47]","[""Right Inferior Temporal Gyrus"": 0.41][""Right Middle Temporal Gyrus"": 0.41][""Right Fusiform Gyrus"": 0.14]","[""Right Middle Temporal Gyrus; posterior division"": 0.75][""Right Inferior Temporal Gyrus; posterior division"": 0.23]"
54,55,247,(0.3;16.3;32.3),"[""None"": 0.83]","[""Cingulum_Mid_L"": 0.32][""Cingulum_Mid_R"": 0.22][""Cingulum_Ant_L"": 0.21][""Cingulum_Ant_R"": 0.20]","[""Left Anterior Cingulate Cortex"": 0.32][""Left Middle Cingulate Cortex"": 0.24][""Right Middle Cingulate Cortex"": 0.22][""Right Anterior Cingulate Cortex"": 0.20]","[""Left Cingulate Gyrus"": 0.44][""Right Cingulate Gyrus"": 0.29][""Right Anterior Cingulate"": 0.13][""Left Anterior Cingulate"": 0.13]","[""Right Cingulate Gyrus; anterior division"": 0.52][""Left Cingulate Gyrus; anterior division"": 0.27][""Left Paracingulate Gyrus"": 0.13]"
58,59,240,(36.7;17.2;3.6),"[""None"": 0.92]","[""Insula_R"": 0.61][""Putamen_R"": 0.20][""Frontal_Inf_Tri_R"": 0.11]","[""Right Insula Lobe"": 0.64][""Right Putamen"": 0.15]","[""Right Insula"": 0.46][""Right Inferior Frontal Gyrus"": 0.40][""Right Claustrum"": 0.14]","[""Right Insular Cortex"": 0.46][""Right Frontal Operculum Cortex"": 0.33]"
63,64,230,(28.8;-0.3;56.8),"[""None"": 1.00]","[""Frontal_Sup_R"": 0.46][""Frontal_Mid_R"": 0.35][""Precentral_R"": 0.20]","[""Right Superior Frontal Gyrus"": 0.39][""Right Middle Frontal Gyrus"": 0.37][""RightPrecentral Gyrus"": 0.23]","[""Right Middle Frontal Gyrus"": 0.80][""Right Superior Frontal Gyrus"": 0.12]","[""Right Superior Frontal Gyrus"": 0.38][""Right Precentral Gyrus"": 0.30][""Right Middle Frontal Gyrus"": 0.28]"
93,94,87,(14.2;-0.5;17.5),"[""None"": 1.00]","[""Caudate_R"": 0.84][""Thalamus_R"": 0.14]","[""Right Caudate Nucleus"": 0.77][""Right Thalamus"": 0.16]","[""Right Caudate"": 0.68][""Right Thalamus"": 0.23]","[""Right Caudate"": 0.71][""Right Thalamus"": 0.26]"


In [ ]:
# ROIS : 
Right middle frontal gyrus(64), Right insula lobe(59), Right temporal pole(32), Right middle temporal gyrus(49), Right Caudate Nucleus(94) 

# Deep Lift

# New Section

In [ ]:
dl_attributions = []
all_folds = pickle.load(open('./data/AllFoldssubjects.pkl', 'rb'))
for fold in range(10):
    fold_weights = torch.load(f'data/Weights/Fold_{fold+1}.pth', map_location=torch.device('cpu'))
    best_clf_model = MTAutoEncoder(tied = False, num_inputs = 19900, num_latent = 512, use_dropout = True) 
    best_clf_model.load_state_dict(fold_weights)
    best_clf_model = best_clf_model.to('cpu')

    test_subjects = all_folds[fold]['test']
    x_asd, y_asd = [], []
    for sample in test_subjects : 
        if(all_corr[sample][1] == 1):
          x_asd.append(all_corr[sample][0])
          y_asd.append(all_corr[sample][1])
    print('Number of ASD subjects in test set : ', len(x_asd))

    x_asd = torch.tensor(x_asd, dtype=torch.float)
    y_asd = np.array(y_asd)  

    y_asd_pred = best_clf_model(x_asd)
    y_asd_pred = y_asd_pred.detach().cpu().numpy()
    y_asd_pred = np.round(y_asd_pred)
    y_asd_pred = np.squeeze(y_asd_pred, axis = 1)

    right_indices = np.where(y_asd_pred == 1)
    x_asd_dl = x_asd[right_indices]
    print('Number of correctly predicted ASD subjects : ', len(x_asd_dl))

    dl_asd = DeepLift(best_clf_model)        
    grads_asd, delta_asd = attribute_image_features(dl_asd, inputs = x_asd_dl, target = 0)                     
    grads_asd = torch.mean(grads_asd, axis = 0)
    grads_asd = grads_asd.detach().cpu().numpy()
    dl_attributions.append(grads_asd)
                            
dl_attributions = np.array(dl_attributions)
# np.save('./data/IG_Attributions.npy', attributions)
print("Attributions shape : ", dl_attributions.shape)

Number of ASD samples in test set :  42
Number of correctly predicted ASD samples :  28


/usr/local/lib/python3.7/dist-packages/captum/_utils/gradient.py:58: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  "required_grads has been set automatically." % index
/usr/local/lib/python3.7/dist-packages/captum/attr/_core/deep_lift.py:323: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  after the attribution is finished"""


Number of ASD samples in test set :  42
Number of correctly predicted ASD samples :  22
Number of ASD samples in test set :  42
Number of correctly predicted ASD samples :  25
Number of ASD samples in test set :  42
Number of correctly predicted ASD samples :  26
Number of ASD samples in test set :  42
Number of correctly predicted ASD samples :  31
Number of ASD samples in test set :  42
Number of correctly predicted ASD samples :  32
Number of ASD samples in test set :  42
Number of correctly predicted ASD samples :  30
Number of ASD samples in test set :  42
Number of correctly predicted ASD samples :  32
Number of ASD samples in test set :  42
Number of correctly predicted ASD samples :  23
Number of ASD samples in test set :  41
Number of correctly predicted ASD samples :  20
Attributions shape :  (10, 19900)


In [ ]:
rois_count = {}    # {roi : number of times it is repeated in all 10 folds}

for grads_asd in dl_attributions :
  
    attr_vals_asd = grads_asd.copy()
    thresh = np.percentile(attr_vals_asd, 99)
    attr_vals_asd = np.where(attr_vals_asd > thresh,  1 , 0) # check1
    corr_matrix_asd = np.zeros((200,200))
    corr_matrix_asd[np.triu_indices(200, 1)] = attr_vals_asd
    print('Number of unique elements in corr_matrix : ', np.unique(corr_matrix_asd, return_counts = True))

    max_sum_rows = np.sum(corr_matrix_asd, axis = 1)      # check 2
    top_indices = np.argsort(max_sum_rows)            # Max value indices
    top_values = max_sum_rows[top_indices]      # Max values

    top20_indices  = top_indices[-20 : ]
    top20_values = top_values[-20 : ]

    print('Most repeated ROIS in ASD (Not index values): ', top20_indices + 1)
    print('Number of times ROIS repeated in ASD : ', top20_values)
    
    for index, roi in enumerate(top_indices): 
        if(roi in rois_count):
            rois_count[roi] += top_values[index]
        else : 
          rois_count[roi] = top_values[index]        

rois_count_sorted = dict(sorted(rois_count.items(), key = lambda x : x[1], reverse = True))
print(rois_count_sorted)

Number of unique elements in corr_matrix :  (array([0., 1.]), array([39801,   199]))
Most repeated ROIS in ASD (Not index values):  [ 89 106 154 130  31  30  43  28  65  21  10  71  32  57  49  59  64 117
  94  55]
Number of times ROIS repeated in ASD :  [3. 3. 3. 4. 4. 4. 4. 4. 4. 4. 5. 5. 5. 5. 6. 6. 6. 6. 7. 8.]
Number of unique elements in corr_matrix :  (array([0., 1.]), array([39801,   199]))
Most repeated ROIS in ASD (Not index values):  [ 78  93  84  33  94  28  71 130  46  49 154  77 106  32  21  59  55  17
  30  64]
Number of times ROIS repeated in ASD :  [3. 3. 3. 3. 3. 3. 4. 4. 4. 4. 4. 5. 5. 5. 5. 5. 6. 6. 6. 7.]
Number of unique elements in corr_matrix :  (array([0., 1.]), array([39801,   199]))
Most repeated ROIS in ASD (Not index values):  [ 66  28 128  15  31  93  94  39  42 105  63  36  17  57  32  49  64  30
  46  59]
Number of times ROIS repeated in ASD :  [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 4. 4. 4. 4. 4. 4. 5. 5. 6.]
Number of unique elements in corr_matrix :  (arra

In [ ]:
delta_asd

tensor([ 0.0448, -0.0280, -0.0142,  0.0200, -0.0104, -0.0270,  0.0045, -0.0501,
         0.0089, -0.0137, -0.0063,  0.0045, -0.0020, -0.0252,  0.0453,  0.0029,
        -0.0170,  0.0187,  0.0781,  0.0047])

In [ ]:
rois = [64, 59, 49, 32, 30, 55, 94, 46]
asd_rois = cc200_labels[cc200_labels['ROI number'].isin(rois)]
display(asd_rois)

In [ ]:
# print(rois_count_sorted).  # Without thresholding and taking the sum. 

{63: 0.28833801775652723, 34: 0.16755825132131255, 14: 0.15259633067985268, 70: 0.13047253297211048, 58: 0.12235610972428701, 41: 0.12204838184740958, 54: 0.11999169962167079, 29: 0.119460891259341, 48: 0.11912324991494355, 52: 0.11543538350486209, 127: 0.11251435212226324, 128: 0.10693960790720322, 88: 0.10345871149475779, 130: 0.10229100258155885, 129: 0.10204925380761753, 90: 0.09465215779146104, 20: 0.09419945080018129, 30: 0.08784245042409317, 83: 0.08576062744973681, 16: 0.08506434713670755, 72: 0.08309108378294525, 105: 0.07935774375355967, 89: 0.07755795605543256, 91: 0.07673209027984447, 82: 0.07601929354820683, 46: 0.07480984718746976, 109: 0.07152370382524624, 19: 0.06990669182987254, 45: 0.06989307003257972, 111: 0.06933357619506557, 118: 0.06875127717805271, 93: 0.06501561961407462, 65: 0.06432255460224846, 163: 0.060248144174904025, 66: 0.05892478769611261, 161: 0.05559441087351655, 110: 0.05502748499986784, 43: 0.05473448984499586, 117: 0.05314646348940971, 116: 0.053124